# Active Learning with Pixano - CIFAR11 Dataset

## initialization of various parameters

### Configuration variables

In [2]:
# The root dir name of the current repo (i.e. pixano or pixano-main etc.)
ROOTDIR='pixano'
# directory where CIFAR11 dataset is downloaded using Notebook "1.convert_dataset_2_explorer.ipynb"
datasets_dir="/home/melissap/Desktop/LAGO/3.githubs/integration/datasets/CIFAR11"
# name of the dataset as defined in "1.convert_dataset_2_explorer.ipynb"
DATASET_NAME="CIFAR11_guns_pixano_v1"
# directory where the raw CIFAR11 dataset will be saved to be transformed latter (images), and also to be used by the active learning auto-annotator (labels)
pixano_datasets_dir='/home/melissap/_pixano_datasets_'
# conda env name builded for running the active learning module as a separate program
customLearnerCondaEnv="customLearner"
# ActiveLearning module's directory
ALModule="ActiveLearning"

### internal experimental variables (that could be defined by the user)

In [3]:
labels_per_round=2000
numInitLabels = labels_per_round
learning_rate=0.001
max_epochs_per_round=100
model_name="resnet18" 
strategy="CoreSet" #i.e. other alternatives may be : EntropySampling #RandomSampling

### External experimental variables

In [4]:
num_rounds = 10

## ... the rest of the notebook should run without any code adjustments/modifications.

> import ROOT dir to import pixano root module , which is the pixano directory

In [5]:
import os
import sys

def insertRootDir(ROOTDIR='pixano'):
    pardir=os.path.dirname(os.path.realpath('__file__'))

    found = False
    potential_root_dir = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__'))))))

    while(os.path.basename(pardir)!=ROOTDIR):

        # print(pardir)
        pardir=os.path.dirname(pardir)
        

        if (os.path.basename(pardir) == ROOTDIR):
            found = True
            break
        if (pardir == "/" ):
            break
    
    if found:
        print("Inserting parent dir : ",pardir)
        sys.path.insert(0,pardir)
        return pardir
    else:
        print(f"ROOTDIR NOT FOUND. You may have to change ROOTDIR variable from : '{ROOTDIR}' to '{potential_root_dir}'")
        return "_NOT_FOUND_"

ROOTDIR = insertRootDir(ROOTDIR)

Inserting parent dir :  /home/melissap/Desktop/LAGO_43integrationDemo/pixano


In [6]:
ALModuleDir = os.path.join(ROOTDIR,ALModule)

In [7]:
from pathlib import Path
from pixano.data import ImageImporter

In [8]:
library_dir=Path(pixano_datasets_dir) # directory where we have install the pixano formatted dataset
import_dir = library_dir / DATASET_NAME

In [10]:
import random
import lancedb
import pyarrow as pa
import numpy as np
import pandas as pd
from PIL import Image
from ActiveLearning.ALearner import (
    Learner,
    BaseAnnotator,
    BaseSampler,
    BaseTrainer,
    getLabels,
    getLabelledIds,
    getUnlabelledIds,
    getTaggedIds,
    getLastRound,
    ddb_str,
    custom_update,
    importTestLabels
)
from pixano.utils import natural_key

In [11]:
from ActiveLearning.customTrainer import customTrainer
from ActiveLearning.customSampler import customSampler

In [12]:
# utility function to convert id (format "<index>.png") to index
def id_to_idx(id: str) -> int:
    return int(id.split(".")[0])
    # return int(id[0:-4])  #remove the last 4 chars (".png")

In [17]:
cifar11_db = lancedb.connect(import_dir)

## Model Trainer Object

We will get raw x_train, x_test, y_test data directly from CIFAR11.

2 proposed Model Trainer Objects, with same model: SimpleTrainer and IncrementalTrainer

In [19]:
def get_CIFAR11(data_dir):

    image_dir = os.path.join(data_dir,"images")
    train_imdir = os.path.join(image_dir,"train")
    test_imdir = os.path.join(image_dir,"test")

    annotation_dir = os.path.join(data_dir,"annotations")
    train_anfile = os.path.join(annotation_dir,"train.csv")
    test_anfile = os.path.join(annotation_dir,"test.csv")

    # read sorted file names     
    X_train = sorted(os.listdir(train_imdir), key=lambda x: int(x.split('.', 1)[0])) 
    X_test = sorted(os.listdir(test_imdir), key=lambda x: int(x.split('.', 1)[0]))
    
    # X_train = np.array([Image.open(os.path.join(train_imdir,x)) for x in X_train])
    # X_test = np.array([Image.open(os.path.join(test_imdir,x)) for x in X_test])

    # # read labels
    Y_train = pd.read_csv(train_anfile).values #.to_numpy(dtype=np.uint8)
    Y_test = pd.read_csv(test_anfile).values #.to_numpy(dtype=np.uint8)

    Y_train = np.array([y[0] for y in Y_train], dtype=np.uint8)
    Y_test = np.array([y[0] for y in Y_test], dtype=np.uint8)

    return (X_train,Y_train),(X_test,Y_test)

(X_train, Y_train), (X_test, Y_test) = get_CIFAR11(datasets_dir)

## Query Sampler Object
<!-- RandomSampler or SequentialSampler -->

#### Custom Trainer

> prepare the directories for data exchange between pixano and annotation tool

In [20]:
import shutil

# TEMPORARY SOLUTION FOR EXCHANGING DATA BETWEEN PIXANO AND AL
def create_dir(path):
    try:
        if (os.path.basename(path)== "temp_data" and os.path.exists(path)):
            shutil.rmtree(path) # erase the previous results
        os.makedirs(path)
    except:
        print(f'Dir {path} exists already')
    return path

# here define the paths of exchanging data between pixano and the customLearner
temp_data_exchange_dir = create_dir(os.path.join(ROOTDIR,"temp_data"))                # define a directory for exchanging data
output_queDir = create_dir(os.path.join(temp_data_exchange_dir,"output_queries"))       # [out] query strategy results
output_accDir = create_dir(os.path.join(temp_data_exchange_dir,"output_accuracy"))      # [out] accuracy results 

## Labeling Interface Objects

Human labeling with Pixano Annotator is built-in, here we specify an Auto Annotator

In [21]:
class AutoAnnotator(BaseAnnotator):
    # custom annotation function
    # as we have ground truth for CIFAR11, we can autofill
    def annotate(self, round):
        candidates = getTaggedIds(self.db, round)
        db_tbl = cifar11_db.open_table("db")
        custom_update(db_tbl, f"id in ({ddb_str(candidates)})", 'label', [str(Y_train[id_to_idx(candidate)]) for candidate in sorted(candidates, key=natural_key)])
        print(f"AutoAnnotator: round {round} annotated.")

## Orchestrator

### Initial Learning

In [22]:
myTrainer = customTrainer(cifar11_db, 
                            DATASET_NAME = DATASET_NAME,
                            output_accDir = output_accDir,
                            import_dir = import_dir,
                            customLearnerCondaEnv = customLearnerCondaEnv,
                            model_name = model_name, 
                            learning_rate = learning_rate, 
                            max_epochs_per_round = max_epochs_per_round,
                            strategy_name = strategy)

mycustomSampler = customSampler(cifar11_db,
                                DATASET_NAME = DATASET_NAME,
                                output_queDir = output_queDir,
                                import_dir = import_dir,
                                customLearnerCondaEnv = customLearnerCondaEnv,
                                model_name = model_name, 
                                strategy_name = strategy, 
                                number_init_labels = numInitLabels, 
                                labels_per_round = labels_per_round,
                                alpha_opt = True)

myTrainer.set_parameter("ALearner",ALModuleDir)
mycustomSampler.set_parameter("ALearner",ALModuleDir)

autofillAnnotator = AutoAnnotator(cifar11_db)

init_learner = Learner(
    db=cifar11_db,
    trainer=myTrainer,
    sampler=mycustomSampler,
    custom_annotator=autofillAnnotator,
    new_al=True,
    verbose=0
)

In [23]:
importTestLabels(cifar11_db,(X_test,Y_test))

Importing validation data in /home/melissap/_pixano_datasets_/CIFAR11_guns_pixano_v1


### Active Learning - Human annotation with Pixano Annotator

We add some auto-annotation rounds

In [24]:
for round in range(num_rounds):
    candidates = init_learner.query(round)
    init_learner.annotate(round)
    init_learner.train(round, epochs="nevermind")

csvQue /home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_0.csv




######################################################## AL ROUND 0 mode query : START #########################################################


Namespace(mode='query', data_name='CIFAR11_guns_pixano_v1', n_label='None', n_init_lb=2000, n_query=2000, data_dir='/home/melissap/_pixano_datasets_/CIFAR11_guns_pixano_v1', pixano_al_module='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning', strategy='CoreSet', round=0, train_out=None, query_out='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_0.csv', seeds=[1, 10, 100, 1000, 10000], init_lb_method='general_random', log_dir='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning/certh_integration/alpha_mix_active_learning/_logs', save_checkpoints=False, save_images=False, print_to_file=False, query_growth_ratio=1, n_drop=5, eps=0.05, max_iter=50, alpha_cap=0.03125, alpha_opt=False, alpha_closed_form_approx=True, alpha_learning_rate=0.1, alpha_clf_coef=1.0, alpha_l2_coef

/home/melissap/miniconda3/envs/customLearner/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/melissap/miniconda3/envs/customLearner/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Round 0 tagged
2000 candidates on round 0
AutoAnnotator: round 0 annotated.


######################################################## AL ROUND 0 mode train : START #########################################################


Namespace(mode='train', data_name='CIFAR11_guns_pixano_v1', n_label='None', n_init_lb=10, n_query=100, data_dir='/home/melissap/_pixano_datasets_/CIFAR11_guns_pixano_v1', pixano_al_module='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning', strategy='CoreSet', round=0, train_out='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_accuracy/accuracy.csv', query_out='path to the file', seeds=[1, 10, 100, 1000, 10000], init_lb_method='general_random', log_dir='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning/certh_integration/alpha_mix_active_learning/_logs', save_checkpoints=False, save_images=False, print_to_file=False, query_growth_ratio=1, n_drop=5, eps=0.05, max_iter=50, alpha_cap=0.03125, alpha_opt=False, al

/home/melissap/miniconda3/envs/customLearner/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/melissap/miniconda3/envs/customLearner/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


...

CIFAR11_guns_pixano_v1
SEED 1
CoreSet
Adam optimizer...
Loading weights resnet18_CIFAR11_guns_pixano_v1_CoreSet_8_weights.pth
 Print the weights of the model to ensure that weights are loaded 
Training started...


  2%|▏         | 2/100 [00:33<27:18, 16.72s/it]

Reached max accuracy at epoch 2 


  2%|▏         | 2/100 [00:49<40:48, 24.99s/it]


Round 0
testing accuracy 0.6420258228768867


######################################################### AL ROUND 9 mode train : END ##########################################################


